In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [4]:
df=pd.read_csv("Bank_Personal_Loan_Modelling.csv")


In [5]:
df.head(10)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.5,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.3,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.6,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0


In [6]:
df.shape

(5000, 14)

In [7]:
df.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [8]:
x=df.iloc[:,df.columns!="Personal Loan"]

In [14]:
y=df.iloc[:,9:10]
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=2)

In [12]:
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=2)

In [15]:
features=['ID', 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']

In [16]:
rf_model.fit(X=x_train[features],y=y_train["Personal Loan"])


RandomForestClassifier(max_features=2, n_estimators=1000, oob_score=2)

In [17]:
print("oob accuracy")
print(rf_model.oob_score_)

oob accuracy
0.9856


In [18]:
for features,imp in zip(features,rf_model.feature_importances_):
    print(features,imp)

ID 0.04954374091206022
Age 0.0422531008298736
Experience 0.04195769938633442
Income 0.340868794446876
Family 0.08777760138285057
CCAvg 0.17425582494670736
Education 0.14613277438225572
Mortgage 0.04321055182158053
Securities Account 0.005693277545670116
CD Account 0.04989836085839516
Online 0.008655689318224198
CreditCard 0.009752584169172096


In [19]:
# income ccavg education are the important independent varible

# build decision tree

In [20]:
tree_model=tree.DecisionTreeClassifier()


In [25]:
tree_model.fit(X=pd.DataFrame(df["Income"]),y=df["Personal Loan"])

DecisionTreeClassifier()

In [26]:
predict=pd.DataFrame([df["Income"],df["CCAvg"],df["Education"]]).T

In [27]:
tree_model=tree.DecisionTreeClassifier(max_depth=6)


In [28]:
tree_model.fit(X=predict,y=df["Personal Loan"]) 


DecisionTreeClassifier(max_depth=6)

In [29]:
with open ("personalloantree.dot","w") as f:
    f=tree.export_graphviz(tree_model,feature_names=["Income","CCAvg","Education"],out_file=f)

In [30]:
tree_model.score(X=predict,y=df["Personal Loan"])

0.9738

In [31]:
test_features=pd.DataFrame([x_test["Income"],x_test["CCAvg"],x_test["Education"]]).T

In [32]:
test_predict=tree_model.predict(X=test_features)

In [33]:
predict_output=pd.DataFrame({"ID":x_test["ID"],"Personal Loan":test_predict})

In [34]:
predict_output.to_csv("output_personalloan.csv",index=False)